# <span style="color:#1111AA">Free spectral range</span>

## Table of Contents

# 1. Overview

## 1.1. Introduction

## 1.2. Prerequisites and reading material

We recommend that you have done the following notebook before you do this one:
* [02_01_01_Resonance_length_tuning](02_01_01_resonance_length_tuning.ipynb)
    * [01_03_01_Finesse](../../01_Introduction/01_03_Finesse/01_03_01_Finesse.ipynb)
    * [01_02_01_Pykat](../../01_Introduction/01_02_Pykat/01_02_01_Pykat.ipynb)
    * [01_01_01_Ipython_notebook](../../01_Introduction/01_01_Ipython_notebook)



or have corresponding knowledge about the longitidinal resonance condition and how it is handled by FINESSE. The links above only works if you started ipython notebook in the top directory of this course.

**Reading material and references:**

[1] [A. Freise, K. Strain, D. Brown, and C. Bond, "Interferometer Techniques for Gravitational-Wave Detection", *Living Reviews in Relativity* **13**, 1 (2010)](http://arxiv.org/pdf/0909.3661v3).

[2] [A. Freise, D. Brown, and C. Bond, "Finesse, Frequency domain INterferomEter Simulation SoftwarE"](http://kvasir.sr.bham.ac.uk/redmine/attachments/download/131/Finesse_2.0_manual.pdf).

[3] [FINESSE syntax reference](http://www.gwoptics.org/finesse/reference/)

## 1.3. After this session you will be able to...

## 2. Free Spectral Range

The distance between two resonance peaks (in Hz) is called the *free spectral range* (FSR) of the cavity. The FSR is often an important property of a cavity since it says how well separated the resonance peaks are. The FSR is related to the width of the peak through the cavity Finesse, which are two concepts that will be handledin the next two notebooks.

The widht of the peak, here defined as the full width at half maximum (FWHM), of a peak is the *linewidth* of a cavity. Use Finesse and Pykat to 'measure' the FSR and linewidth of this example cavity.

The distance between two resonance peaks (in Hz) is called the *free spectral range* (FSR) of the cavity. The width (full width at half maximum) of a peak is the *linewidth* of a cavity. Use Finesse and Pykat to 'measure' the FSR and linewidth of this example cavity.

Tips: 
  * for the FSR computation, use the function `scipy.optimize.minimize_scalar` to find two peaks of the transmitted power
  * for the linewidth computation, use the function `scipy.optimize.bisect` to find the point at which the transmitted power is half the maximum transmitted power
  * each of the scipy functions require as their first argument a function name, so first define a function, e.g. like this (not exactly in this form, you need to do a small change for your tasks):
```python
def my_function(x):
        kat.laser.f = x
        out=kat.run()
        return out['trans']
```

In the Finesse code you can replace the `xaxis` command with `noxaxis` so that a call to `kat.run()` returns only one value.

And then you can call the scipy functions, e.g. as:

```python
import scipy
scipy.optimize.minimize_scalar(my_function, ...)
```

We use the example above and just replace the `xaxis` command with `noxaxis`:

In [3]:
kat=finesse.kat() # initialising Finesse
kat.verbose = False
code = """
l laser 2 0 n0  # a laser with 1W of power at the default wavelength of 1064nm   
s s1 1 n0 n1 # empty space of 1m length
bs bs1 0.5 0.5 0 0 n1 n2 n3 n4
s s2 1 n3 nc1

m m1 0.8 0.2 0 nc1 nc2
s sL 3000 nc2 nc3
m m2 0.9 0.1 0 nc3 nc4

pd trans nc4
pd refl n4

noxaxis 

yaxis abs 
"""
kat.parseKatCode(code) # passing the input text to the Finesse object
out = kat.run() # run Finesse, the output will be stored in 'out'

NameError: name 'finesse' is not defined

We need to import the scipy package:

In [2]:
import scipy.optimize

We define a function which returns the transmitted power as a function of laser frequency:

In [ ]:
# defining a function we can use in a optimiser for peak finding
def kat_frs(x):
    kat.laser.f = x
    out=kat.run()
    return -1.0*out['trans']

The small change to the definition given as a tip above is the minus sign in the `return` statement: It was suggested to use the function `scipy.optimize.minimize_scalar`, which can only *minimise*. Therefore, in order to find peaks we have to define our function such that the peaks become negative dips. We do a quick test if the function works:

In [ ]:
kat_frs(10e3) # testing if it works

And now we can use scipy to find two peaks. We only need to provide approximate ranges in which the peaks should be found.

(The functions below will give a warning about tolerances that we can ignore.)

In [ ]:
# Computing FRS
res1=scipy.optimize.minimize_scalar(kat_frs, bounds=(-4, 7), method='bounded',tol=1e-10)
res2=scipy.optimize.minimize_scalar(kat_frs, bounds=(-60000, -40000), method='bounded',tol=1e-10)
FSR=res1.x-res2.x
print ("The free spectral range of the cavity is FSR = {0:.0f} Hz".format(FSR))

In order to measure the linewidth we have to use a slightly different method. The linewidth 'full width at half maximum' (FWHM) is defined as two times the frequency difference between the peak and the point at which the height of the peak has dropped to 0.5 the maximum.

To find that point we can use a root finding algorithm. We need to define a new function that has a root (zero-crossing) at the 'power equals half maximum' point:

In [ ]:
# For the linewidth, we define a new function we can use in a optimiser for root finding
peak_value=-1.0*res1['fun']
def kat_line(x):
    kat.laser.f = x
    out=kat.run()
    return out['trans']-0.5*peak_value # this function is zero when out = 0.5 * max(out)

In [ ]:
kat_line(500) # testing

And now we use the `bisect` method to find the exact frequency at which the zero occurs. We knoe the peak is at 0 Hz, so the linewidth can be computed as twice the zero-crossing frequency:

In [ ]:
# using the bisection method to find the root:
(lw, info)=scipy.optimize.bisect(kat_line,1.0,10000.0, xtol=1e-8, maxiter=500, full_output=True)
print("The cavity linewidth (FWHM) is = {0:.0f} Hz".format(2*lw)) # we know the peak is at 0 Hz